# Problem 1 

contrat = general; dynamics = local volatility, pricer = tree. 

In [1]:
import numpy as np

In [2]:
class localvolDynamics:

    def __init__(self, S0, r, q, maxvol, localvol):
        self.S0 = S0
        self.r = r
        self.q = q
        self.maxvol = maxvol
        self.localvol = localvol


In [3]:
hw2dynamics = localvolDynamics(S0 = 100, r = 0.06, q = 0.01, maxvol = 0.6,
                     localvol = lambda S,t: np.minimum(0.2+5*np.log(S/100)**2+0.1*np.exp(-t), 0.6))

In [5]:
class CallOnAmericanPut:

    def __init__(self, putexpiry, putstrike, callexpiry, callstrike):
        self.putexpiry = putexpiry
        self.putstrike = putstrike
        self.callexpiry = callexpiry
        self.callstrike = callstrike


In [6]:
hw2contract = CallOnAmericanPut(putexpiry=0.75, putstrike=95, callexpiry=0.25, callstrike=10)

In [102]:
class TreeEngine:

    def __init__(self, N):
        self.N = N

        self.deltat = None 
        self.rGrow = None 
        self.deltax = None 
        
        self.put_price = {}
        self.pu = {}
        self.pd = {} 
        self.pm = {}

    def American_put_price(self, contract, dynamics):
        Sgrid = dynamics.S0*np.exp(np.linspace(self.N, -self.N, num=2*self.N+1, endpoint=True)*self.deltax) #last period S 
        optionprice = np.maximum(contract.putstrike - Sgrid, 0)
        

        for t in np.linspace(self.N-1, 0, num=self.N, endpoint=True)*self.deltat:
            Sgrid = Sgrid[1:-1]
            nu = self.rGrow - 0.5*(dynamics.localvol(Sgrid,t)**2)        
            Pu = 0.5*((dynamics.localvol(Sgrid,t)**2 * self.deltat + nu**2 * self.deltat**2)/(self.deltax**2)+(nu*self.deltat)/(self.deltax))       
            Pd = 0.5*((dynamics.localvol(Sgrid,t)**2 * self.deltat + nu**2 * self.deltat**2)/(self.deltax**2)-(nu*self.deltat)/(self.deltax))       
            Pm = 1 - Pu - Pd  

            continuation_value = np.exp(-dynamics.r * self.deltat) * (
                optionprice[:-2] * Pu + 
                optionprice[1:-1] * Pm + 
                optionprice[2:] * Pd
            )
     
            intrinsic_value = np.maximum(contract.putstrike - Sgrid, 0)

            optionprice = np.maximum(intrinsic_value, continuation_value)

            self.put_price[t] = optionprice 
            self.pu[t] = Pu
            self.pd[t] = Pd
            self.pm[t] = Pm 

        return optionprice[0]
    
    def European_call_price(self,contract,dynamics):
        optionprice= np.maximum(self.put_price[contract.callexpiry] - contract.callstrike, 0)

        call_N = int(contract.callexpiry / self.deltat)

        for t in np.linspace(call_N-1, 0, num=call_N, endpoint=True)*self.deltat:
            Pu = self.pu[t]
            Pd = self.pd[t]
            Pm = self.pm[t]
            optionprice = np.exp(-dynamics.r * self.deltat) * (
                optionprice[:-2] * Pu + 
                optionprice[1:-1] * Pm + 
                optionprice[2:] * Pd
            )
        return optionprice[0]


    def price_compound_localvol(self, contract, dynamics):

        self.deltat = contract.putexpiry / self.N
        sigma_avg = dynamics.localvol(dynamics.S0,0)
        self.deltax = np.maximum(sigma_avg*np.sqrt(3*self.deltat), dynamics.maxvol*np.sqrt(3*self.deltat)) 
        self.rGrow = dynamics.r - dynamics.q
    

        price_of_put = self.American_put_price(contract,dynamics)     

        price_of_call_on_put =  self.European_call_price(contract, dynamics)  

        return (price_of_put, price_of_call_on_put)


In [103]:
hw2tree = TreeEngine(N=30000) 

In [104]:
(answer_part_a, answer_part_b) = hw2tree.price_compound_localvol(hw2contract,hw2dynamics)

In [105]:
(answer_part_a, answer_part_b)

(7.006887597204046, 1.5922137120613111)

Part (a): put price is 7.006. 


Part (b): call price is 1.592.

# Problem 2 

## (a)

$$
f = S_0N(d_1) - Ke^{-rT}N(d_2)\\
d_{1,2} = \frac{log(S_0e^{rT}/K)}{\sigma\sqrt{T}} \pm \frac{\sigma\sqrt{T}}{2}\\
\text{First order of Talyor approximation: } f(x) = f(x_0) + f'(x_0) * (x-x_0) + O(x- x_0)^1 \\

delta = \frac{\partial f}{\partial S} = N(d_1) = \frac{rT}{\sigma\sqrt{T}} + \frac{\sigma\sqrt{T}}{2} \\
\text{For small |x|: }\\
delta(x) = N(0) + N'(0) * (x) + O(0) = 0.5 + \frac{1}{\sqrt{2\pi}} * (\frac{rT}{\sigma\sqrt{T}} + \frac{\sigma\sqrt{T}}{2})
$$

$$
\sigma = 0.2, T = 0.25, r = 0.01 \\
delta(x) \approx 0.53 
$$
The delta approximation to two decimal places is 0.53.

## (b)

Second Order Talyor's Expansion: 

$$
C(S) = C(S_0) + \frac{\partial C}{\partial S}(S - S_0) + \frac{1}{2}\frac{\partial^2 C}{\partial S^2}(S - S_0)^2 + O(S_0)^3\\
=  C(S_0) + \text{dollar delta }* \frac{1}{S_0}*(S-S_0) + \text{dollar gamma }* \frac{50}{S_0^2}*(S-S_0)^2\\
$$

$$
C(S) = 5 + 3 * (3.6- 4)/4 + 0.02 *(3.6-4)^2 * 50/ 4^2 = 4.71
$$